# Hybrid systems in QuantNBody: Philosophy of the package

Lucie Pepe - - Laboratoire de Chimie Quantique de Strasbourg, France - January 2024 

In [1]:
# Import librairies
import numpy as np 
import psi4     
import math
import scipy 
import matplotlib.pyplot as plt
import quantnbody as qnb

<img src="Intro.png" width="700"  style="margin:auto"/>

<img src="Intro_2.png" width="700"  style="margin:auto"/>

The simulation of hybrid light-matter systems requires efficient numerical tools capable of taking into account the subtleties generated by the interaction between bosonic and fermionic particles. This type of model can be easily simulated with QuantNBody (see Tuto_Holstein_and_polaritonic_QED_dynamics.ipynb available on the website).

To perform such simulations in Quantnbody, one needs to numerically build an hybrid Hilbert space as a tensor product of bosonic and fermionic Hilbert spaces, where the hybrid many-body configuration states will be expressed as:
$$ | \Phi_{hybrid} \rangle = | \Phi_{bos} \rangle \otimes | \Phi_{elec} \rangle $$

where $| \Phi_{bos} \rangle$ and $| \Phi_{elec} \rangle$ represent the bosonic and fermionic occupancy number vectors respectively. Here, a fermionic occupancy number $| \Psi_{bos} \rangle$ describes occupied fermionic molecular orbitals (with a fixed number of particles) interacting with occupied bosonic modes (such as photons, phonons, etc.,). The latter are then considered as forming an environment (where the number of particles is not conserved) for the fermionic system. 

**The following question naturally arise: How to generate all the associated hybrid states to build such an hybrid many-body basis ?** 

### Step 1: How to build an hybrid many-body basis ? 

**Reminder on fermionic and bosonic many-body basis in QuantNBody**

As shown in the previous tutorials (See Tutorial Tuto_FIRST_STEP.ipynb available on the website), a purely fermionic or purely bosonic system can be generated using the QuantNBody package by providing a list of many-body states which describes the repartition of $N_{elec}$ electrons in $2N_{MO}$ spin-orbitals, or the repartition of $N_{b}$ bosons in $N_{modes}$ modes. These states are numerically referenced by a list of kappa indices such that :
$$
\Big\lbrace |\kappa \rangle \Big\rbrace_{\textstyle \kappa=1}^{\textstyle \dim_{\mathcal{H}_{bos/elec}}}
$$ 

- which describes the repartition of $N_{elec}$ electrons in $2N_{MO}$ spin-orbitals in a fermionic Hilbert space  $\mathcal{H}_{elec}$ of dimension :
$$\dim_{H_{elec}} = \binom{2N_{MO}}{N_{elec}}$$

- or which describes the repartition of $N_{b}$ bosons in $N_{modes}$ modes in a bosonic Hilbert space $\mathcal{H}_{bos}$ of dimension :
$$\dim_{H_{B}} = \binom{N_{b} + N_{mode} - 1 }{N_{mode}}$$

**Now, the remaining question is the following : How can we use these two sub-basis to numerically construct an hybrid one ?**

Considering that the number of bosonic particles is no longer conserved in the Fock bosonic space ${\mathcal{F}_{bos}}$, one needs to compute all the possible fermion-boson configurations of $\mathcal{H}_{hybrid}$ by taking into account :

-  all the possible fermionic determinants in $\mathcal{H}_{elec}$
-  all the possible repartition of the $N_{b}^{max}$ bosons, starting from the minimal value $N_b = 0 \rightarrow N_{b}^{max} $, in the Fock bosonic space ${\mathcal{F}_{bos}}$ 

The dimension of the hybrid Hilbert space will thus become : 
$$\dim_{\mathcal{H}_{hybrid}} =  \dim_{\mathcal{F}_{bos}} \times \dim_{\mathcal{H}_{elec}} $$


**Exemple :**
We take here the case where we have $N_{elec} =2 $ fermions, that can take place in $2 N_{MO}$ spin-orbitals, interacting with $N_{mode} = 2$ bosonic modes, where the maximal number of bosons in the system is $N_{b}^{max} = 2$. Let's see how QuantNbody expresses the associated hybrid many-body basis : 

In [2]:
# ======================================
# Define the fermionic system
# ======================================
N_elec =2 # number of fermions 
N_MO = 2 # number of molecular orbitals 

# ======================================
# Define the bosonic system
# ======================================
N_b_max = 2 # maximal number of bosons in the whole bosonic system 
N_mode = 2 # number of bosonic modes 
list_bosons = range(N_b_max+1) # list of all possible number of bosons that can be distributed in the bosonic modes  

# ======================================
# Build the hybrid many-body basis
# ======================================
nbody_basis = qnb.hybrid_fermionic_bosonic.tools.build_nbody_basis(N_mode, list_bosons, N_MO, N_elec) 

# Print results
print('Shape  of the hybrid kappa states')
for s in range(len(nbody_basis)):
    print('| kappa={} >'.format(s), '=', nbody_basis[s])
    

Shape  of the hybrid kappa states
| kappa=0 > = [0 0 1 1 0 0]
| kappa=1 > = [0 0 1 0 1 0]
| kappa=2 > = [0 0 1 0 0 1]
| kappa=3 > = [0 0 0 1 1 0]
| kappa=4 > = [0 0 0 1 0 1]
| kappa=5 > = [0 0 0 0 1 1]
| kappa=6 > = [1 0 1 1 0 0]
| kappa=7 > = [1 0 1 0 1 0]
| kappa=8 > = [1 0 1 0 0 1]
| kappa=9 > = [1 0 0 1 1 0]
| kappa=10 > = [1 0 0 1 0 1]
| kappa=11 > = [1 0 0 0 1 1]
| kappa=12 > = [0 1 1 1 0 0]
| kappa=13 > = [0 1 1 0 1 0]
| kappa=14 > = [0 1 1 0 0 1]
| kappa=15 > = [0 1 0 1 1 0]
| kappa=16 > = [0 1 0 1 0 1]
| kappa=17 > = [0 1 0 0 1 1]
| kappa=18 > = [2 0 1 1 0 0]
| kappa=19 > = [2 0 1 0 1 0]
| kappa=20 > = [2 0 1 0 0 1]
| kappa=21 > = [2 0 0 1 1 0]
| kappa=22 > = [2 0 0 1 0 1]
| kappa=23 > = [2 0 0 0 1 1]
| kappa=24 > = [1 1 1 1 0 0]
| kappa=25 > = [1 1 1 0 1 0]
| kappa=26 > = [1 1 1 0 0 1]
| kappa=27 > = [1 1 0 1 1 0]
| kappa=28 > = [1 1 0 1 0 1]
| kappa=29 > = [1 1 0 0 1 1]
| kappa=30 > = [0 2 1 1 0 0]
| kappa=31 > = [0 2 1 0 1 0]
| kappa=32 > = [0 2 1 0 0 1]
| kappa=33 > = [0 2

**What is the meaning of these 36 list of numbers ?**

Here, each bit string represents an hybrid many-body state. As an example, let's check the first state for which we have :
$$| \kappa  = 0\rangle = | \underbrace{0}_{\substack{\textstyle{ 1st }\\ \textstyle{ mode}}}, \; \; \;\underbrace{0}_{\substack{\textstyle{ 2nd}\\ \textstyle{ mode}}},\;\underbrace{   \overbrace{1}^{ \textstyle  {\alpha}}, \; \; \;\overbrace{1}^{ \textstyle  {\beta}},}_{\textstyle 1st \ MO}\; \; \underbrace{\overbrace{0}^{ \textstyle  {\alpha}}, \; \; \; \overbrace{0}^{ \textstyle  {\beta}}}_{\textstyle 2nd \ MO} \rangle$$

Here we choose to structure the occupation numbers as follows: 

- Bosonic modes are expressed at the beginning of the list of numbers. Each value refers to the number of bosons in the associated bosonic mode.
- For the following fermionic part, each couple of terms refer to **a same spatial orbital**, where **even** indices refer to **$\alpha$-spinorbitals**, and **odd** indices to **$\beta$-spinorbitals**.

Considering the 36 list of numbers given here, we see that the first set of 6 list of numbers contains all the possible fermionic configurations, for one bosonic configuration. The 6 following list of numbers contain also all possible fermionic configurations, for another bosonic one, and so on... until all the possible repartitions of the $N_b = 0 \rightarrow N_{b}^{max} $ bosons in the $ N_{mode}$ bosonic modes has been scanned, and until the number of list of numbers reaches the $\dim_{\mathcal{H}_{hybrid}}$ value. 


**The following question then arises : How to build fermionic, bosonic and fermion-boson operators in this hybrid many-body basis ?** 

### Step 2: About building operators in the hybrid many-body basis

From a mathematical point of view, all operators conserving the number of fermionic particles can be decomposed as a serie of $\hat{a}^\dagger \hat{a}$ operators. Conversely, all the operators that are not conserving the number of bosonic particles must be expressed in terms of a serie of creation $\hat{b}$ and annhilation $\hat{b}^\dagger$ operators. We will here focus on these 3 operators. Each of these operators needs to be defined in the hybrid Hilbert space $\mathcal{H}_{hybrid}$, i.e., the 3 operators must be defined such that : 

- the fermionic operator must only act on the fermionic part and remains the bosonic part unchanged :
  $\hat{a}^\dagger \hat{a} \otimes \mathbb{1}_{bos}$
- the bosonic operators must only act on the bosonic part and remain the fermionic part unchanged :
  $ \hat{b} \otimes \mathbb{1}_{elec}$ , and $ \hat{b}^\dagger \otimes \mathbb{1}_{elec}$

**Quick reminder about operators in QuantNBody**

The operators here will be expressed exactly like in a purely fermionic or purely bosonic system. Meaning that for each configuration, we associate a unique $\kappa$ index which defines a unique "numerical" vector. In practice, any numerical representation of a given many-body operator will be given in the basis indexed by the $\kappa$. As an example, let us imagine we want to encode numerically a second quantization operator $\hat{O}$. This means in practice that we create a matrix representation in the hybrid many-body basis such that

$$ \hat{O} = \sum_{\kappa, \kappa' 
 =1}^{\dim_H}  \langle \kappa' | \hat{O} | \kappa  \rangle  \; | \kappa'    \rangle\langle \kappa |  $$

In practice, this indexing is realized by the QuantNBody package and used then as a central tool to build every matrix element of a given many-body operators.

### Step 3: Build the fermionic $\hat{a}^\dagger \hat{a}$ operator in the hybrid basis

In the hybrid part of the package, the fermionic operator $\hat{a}^\dagger_{p,\sigma} \hat{a}_{q,\tau}$ is written such as in the fermionic part of the package, meaning that the implemented function of creation of the operator is strictly the same...

 **See tutorial "FIRST STEP"**  : 
*The way each operator is stored follows the way we order the spin-orbitals in our many-body states. As an illustrative example, taking the following elements will return the associated many-body operators:*

<center>  a_dagger_a[0,0]  $ \longrightarrow \hat{a}^\dagger_{0,\alpha} \hat{a}_{0,\alpha}$ </center>

<center>  a_dagger_a[1,0]  $ \longrightarrow \hat{a}^\dagger_{0,\beta} \hat{a}_{0,\alpha}$ </center>

<center>  a_dagger_a[10,1]  $ \longrightarrow \hat{a}^\dagger_{5,\alpha} \hat{a}_{0,\beta}$ </center>

*In practice, the resulting many-body operators we get access to are still expressed in the original many-body basis stored under a sparse format.*

... Except that now, it will only act on the fermionic degrees of freedom of the full hybrid-manybody basis. The operator $\hat{a}^\dagger_{p,\sigma} \hat{a}_{q,\tau}$ will thus be a matrix of shape $(2N_{M0},2N_{M0})$, i.e., the fermionic degrees of freedom, where each element is a sparse matrix with a shape $(\dim_{\mathcal{H}_{hybrid}},\dim_{\mathcal{H}_{hybrid}})$, containing the encoding of all the hybrid determinants $| \kappa  \rangle $ in the whole configuration basis. Thus, each sparse matrix of each elements of this operator will contain only transformations from one configuration to another, while the bosonic part remains unchanged. 


**Example of some elements of the $\hat{a}^\dagger_{p,\sigma} \hat{a}_{q,\tau}$ operator:**

If we take again the same system as in the previous example, one can construct directly the associated fermionic operator with the encoded function : 

In [3]:
a_dagger_a = qnb.hybrid_fermionic_bosonic.tools.build_fermion_operator_a_dagger_a(nbody_basis, N_mode)

If we look at the element a_dagger_a[0,2], we get acces to the squared sparse matrix, whose size is related to the size of the manybody basis, which shows the promotion of 1 fermion operator from the first spin orbital (first MO, spin up, with index 0) to the third spin orbital (second MO, spin up, with index 2) of the fermionic sub-system:

In [4]:
print(a_dagger_a[0,2])

  (0, 3)	-1.0
  (2, 5)	1.0
  (6, 9)	-1.0
  (8, 11)	1.0
  (12, 15)	-1.0
  (14, 17)	1.0
  (18, 21)	-1.0
  (20, 23)	1.0
  (24, 27)	-1.0
  (26, 29)	1.0
  (30, 33)	-1.0
  (32, 35)	1.0


We observe here that the action of the a_dagger_a operator (promotion of one fermion from spin orbital 1 to spin orbital 3) is only possible from one specific configuration (an encoded $| \kappa  \rangle $ here) to another one :

- The first line shows that we have the possibility to do it from the $| \kappa  \rangle $ state $|0  \rangle $ to $| 3  \rangle $. If we look at the hybrid many-body basis printed just before, we clearly see that there is no change in the occupation number of the bosonic modes between these two states, i.e., that the bosonic modes as well beeing ignored by the fermionic operator. Moreover, if we look at the fermionic occupation numbers of the associated $| \kappa  \rangle $ states, we do see that we have the promotion of a fermion from the first spin orbital (third term in the bitstring) to the third spin orbital (fifth term in the bitstring).
- Finally, action "-1" refers to electron unpairing in the starting MO under consideration, while action "+1" refers to electron pairing in the target MO under consideration.  

### Step 3: Build the bosonic $\hat{b}$ and $\hat{b}^\dagger$ operators in the hybrid basis

The same idea is applied to the construction of the bosonic operator, i.e. we construct bosonic annihilation and creation operators, defined in the hybrid many-body basis, that act only on bosonic modes while leaving fermionic occupation numbers unchanged. In contrast to the fermionic counting operator $\hat{a}^\dagger \hat{a}$, and as the hybrid many-body basis is constructed by considering that the number of bosonic particles is not conserved, one can construct directly the bosonic creation operator on $\hat{b}$, by finding the $| \kappa ' \rangle $ state associated to a $| \kappa  \rangle $ state, where a boson has been added. 

This definition of the $\hat{b}$ bosonic creation operator has been directly encoded in QuantNBody, by using exactly the same $| \kappa \rangle $ mapping as in the fermionic part, except that we obviously use the bosonic commutation and changes of phases rules instead of the fermionic ones as follows : 

$\hat{b}^\dagger |1,n \rangle = \sqrt{n+1} |1,n+1 \rangle $
$\hat{b} |1,n \rangle = \sqrt{n} |1,n-1 \rangle $


**Example of some elements of the $\hat{b}$ operator:**

In [5]:
# We compute here the b operator
b = qnb.hybrid_fermionic_bosonic.tools.build_boson_anihilation_operator_b(nbody_basis,N_mode)

# Shape of the b operator ? 
print(np.shape(b))

(2,)


If we look at the shape of the $\hat{b}$ operator, we see that the $\hat{b}$ operator has only 2 elements, as we have defined the system with 2 bosonic modes. 
Knowing that the action of a $\hat{b}$ operator is the addition of a phase $\sqrt{N}$, what if we look then at the sparse.matrix with the shape $(\dim_{\mathcal{H}_{hybrid}},\dim_{\mathcal{H}_{hybrid}})$ of the first element of $\hat{b}$, associated to the first bosonic mode ?

In [6]:
print(b[0])

  (0, 6)	1.0
  (1, 7)	1.0
  (2, 8)	1.0
  (3, 9)	1.0
  (4, 10)	1.0
  (5, 11)	1.0
  (6, 18)	1.4142135623730951
  (7, 19)	1.4142135623730951
  (8, 20)	1.4142135623730951
  (9, 21)	1.4142135623730951
  (10, 22)	1.4142135623730951
  (11, 23)	1.4142135623730951
  (12, 24)	1.0
  (13, 25)	1.0
  (14, 26)	1.0
  (15, 27)	1.0
  (16, 28)	1.0
  (17, 29)	1.0


If we look at the associated $(| \kappa  \rangle ,| \kappa ^{\prime} \rangle )$ elements of each line : 

For the first line, we see that there is an action of adding a boson from $| \kappa  \rangle$ state $| 0  \rangle$ to $| 6  \rangle$, which makes the adding of a phase in $\sqrt{n}$ on this element, i.e. the value 1 shown here, as $n =1$ refers to the number of bosons in the target mode under consideration (we switch from 0 boson to 1 boson). If we look at the associated $| \kappa  \rangle$ and $| \kappa ' \rangle$ states in the hybrid many-body basis, we do see that there is no change in the fermionic part between these two determinants, but that the number of bosons in the bosonic mode as well been increased from 0 to 1. 

This can be easily verified using the functions implemented in the package : the visualize_wft function allows to see the bitstring as a tensor product between the bosonic and the fermionic determinants. 

In [13]:
# ======================================
# Initial state |0>
# ======================================

# 1) Choose the kappa state to analyze 
initial_many_body_state = [0,0,1,1,0,0]
# 2) Obtain the qnb traduction in the hybrid many-body basis   
initial_state =  qnb.hybrid_fermionic_bosonic.tools.my_state(initial_many_body_state, nbody_basis)
# 2) Visualize the associated wavefunction 
print( 'Initial state :')
qnb.hybrid_fermionic_bosonic.tools.visualize_wft(initial_state,
                                                 nbody_basis, 
                                                 N_mode, 
                                                 cutoff=0.005)

# ======================================
# Final state |6> 
# ======================================

# 1) Choose the kappa' state to analyze 
final_many_body_state = [1,0,1,1,0,0]
# 2) Obtain the qnb traduction in the hybrid many-body basis   
final_state =  qnb.hybrid_fermionic_bosonic.tools.my_state(final_many_body_state, nbody_basis)
# 2) Visualize the associated wavefunction 
print( 'Final state :')
qnb.hybrid_fermionic_bosonic.tools.visualize_wft(final_state,
                                                 nbody_basis, 
                                                 N_mode, 
                                                 cutoff=0.005)


Initial state :

	-----------
	 Coeff.     N-body state and index 
	-------     ----------------------
	+1.00000   |00⟩ ⊗ |1100⟩    #0 

Final state :

	-----------
	 Coeff.     N-body state and index 
	-------     ----------------------
	+1.00000   |10⟩ ⊗ |1100⟩    #6 



'\n\t-----------\n\t Coeff.     N-body state and index \n\t-------     ----------------------\n\t+1.00000   |10⟩ ⊗ |1100⟩    #6 \n'

**About the $\hat{b}^\dagger$ operator:**

Finally, the associated $\hat{b}^\dagger$ operator can directly be obtained by taking the transpose of $\hat{b}$. 

In [8]:
b_dag= []
for mode in range(N_mode):
    b_dag += [b[mode].T]

In [9]:
print(b_dag[0])

  (6, 0)	1.0
  (7, 1)	1.0
  (8, 2)	1.0
  (9, 3)	1.0
  (10, 4)	1.0
  (11, 5)	1.0
  (18, 6)	1.4142135623730951
  (19, 7)	1.4142135623730951
  (20, 8)	1.4142135623730951
  (21, 9)	1.4142135623730951
  (22, 10)	1.4142135623730951
  (23, 11)	1.4142135623730951
  (24, 12)	1.0
  (25, 13)	1.0
  (26, 14)	1.0
  (27, 15)	1.0
  (28, 16)	1.0
  (29, 17)	1.0


We obtain exactly the opposite of the results obtained for the $\hat{b}$ operator.

**About the counting $\hat{b}^\dagger \hat{b}$ operator:**

Once $\hat{b}$ is buit, one can use the operators as building blocks for a wide possibilty of operators such as the $\hat{b}^\dagger \hat{b}$ counting one. As an exemple, let's count the number of bosons in the second mode of one state : 

In [14]:
# ==================================================================================
# Let's take the | kappa=30 > = [0 2 1 1 0 0] state with 2 bosons in the second mode 
# ==================================================================================

# 1) Define the state of interest 
test_state = [0,2,1,1,0,0]

# 2) Encode it in the qnb hybrid many_body basis 
test_state_qnb =  qnb.hybrid_fermionic_bosonic.tools.my_state(test_state, nbody_basis)

# 3) Let's count the number of bosons in the second mode (with index 1)
occ_number = test_state_qnb.T @ (b_dag[1] @ b[1]) @ test_state_qnb
print(occ_number)

2.0000000000000004
